In [2]:
"""
    This notebook captures the process by which de-identification is ran against the AoUS bigquery database
    @TODO:
    - Need to add some form of logs to know what is going on at all times
    - 
"""
from __future__ import division
import json
from google.cloud import bigquery as bq

from deid import *
client = bq.Client.from_service_account_json('/home/steve/git/rdc/deid/config/account/account.json')
Orchestrator(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'],dataset='raw',table='person',fields=['value_as_number'])
import sys
del sys.modules['deid']

SELECT a.person_id,a.gender_concept_id,a.year_of_birth,a.month_of_birth,a.day_of_birth,a.race_concept_id,a.ethnicity_concept_id,a.location_id,a.provider_id,a.care_site_id,a.person_source_value,a.gender_source_value,a.gender_source_concept_id,a.race_source_value,a.race_source_concept_id,a.ethnicity_source_value,a.ethnicity_source_concept_id,birth_datetime FROM (
                    SELECT person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
                    FROM raw.person
                ) a INNER JOIN (
                        SELECT x.person_id,
                DATE_DIFF( CAST(x.value_as_string AS DATE), CAST(__targetTable.birth_datetime AS DATE), DAY) as birth_datetime
             
                        FROM raw.observation x INNER J

In [ ]:
import os
import pandas as pd
from datetime import datetime

ids = [329225,562270]
path = '/home/steve/git/rdc/deid/config/account/account.json'
os.environ['PANDAS_GBQ_CREDENTIALS_FILE']  = path
df = pd.read_gbq('select * from raw.observation where person_id in( 562270,329225)',project_id='aou-res-deid-vumc-test',private_key=path)
df[list(set(df.columns) - set(['value_as_number']))]
p = df.filter(items = ['person_id','observation_source_value','value_as_string'])
p = p.query("observation_source_value == 'ExtraConsent_TodaysDate'" )
p
# date(p['value_as_string'][:1].values[0])
# df 

In [ ]:
# from import datetime import datetime
value="1985-05-07"
datetime.strptime(value,"%Y-%m-%d")
client = bq.Client.from_service_account_json(path)

f = open(path)
account= json.loads(f.read())

f.close()
#r = client.query("select * from raw.person limit 1",location='US')

# job.error_result
p = pd.read_gbq("select * from raw.observation limit 1",project_id=account['project_id'],private_key=json.dumps(account))
dtypes = p.ftypes 
dtypes
date_fields = set(['date','datetime','timestamp'])
[name for name in p.columns if (set(dtypes[name].replace(':dense','').replace('64','').split('[ns]')) & date_fields) or (set(name.split('_')) & date_fields)  ]
# dtypes['person_id'].split(':')

In [ ]:

#
# This section performs suppression of PPI on a meta-table
# @TODO INsure the way in which the PPI are identified is correct. You should make sure the dates aren't suppressed.
# Dates will have to be shifted ()
#
handler = SuppressPPI(client=client)
r = handler.can_do('raw')
handler.do(i_dataset='raw',table_name='observation', o_dataset='out')

In [ ]:
#
# This section performs a suppression of fields from a non-meta table
# A configureation should be provided to the class
#
config = {"raw":{"person":['year_of_birth','month_of_birth','day_of_birth']}}
h = SuppressEHR(client=client,config = config)
h.can_do('raw','person')
h.do(i_dataset='raw',o_dataset='out',table_name='person')

In [ ]:
#
# This section will perform a date shifting on PPI and EHR
# The date shifting operation will (to a certain degree) include suppression
#
h = Shift(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'])
[h.vocabulary_id,h.concept_sql]
r = h.can_do('raw','observation')
h.get('raw','observation')['fields']
# h.do(i_dataset='raw',table_name='observation',o_dataset='out')
# h.sql

In [ ]:
# ref = client.dataset('raw').table('person')
# info = client.get_table(ref).schema
# [field.field_type for field in info]
h = DropFields(client=client)
h.can_do('raw','observation')

# str(h.policies['raw.observation']).replace('\n','')
# h.get('raw','person')

# fields = h.policies['raw.concept']
# fields
# str(fields.replace('\n','').replace('  ',' '))

In [ ]:
Orchestrator(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'],dataset='raw',table='observation',fields=['person_id','value_as_number'])